In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/trec-2025/2025-starter-kit-main/.example.env
/kaggle/input/trec-2025/2025-starter-kit-main/main.py
/kaggle/input/trec-2025/2025-starter-kit-main/README.md
/kaggle/input/trec-2025/2025-starter-kit-main/produce_run.py
/kaggle/input/trec-2025/2025-starter-kit-main/output/tracking_data.json
/kaggle/input/trec-2025/2025-starter-kit-main/output/dragun-organizers-starter-kit-task-1
/kaggle/input/trec-2025/2025-starter-kit-main/output/dragun-organizers-starter-kit-task-2
/kaggle/input/trec-2025/2025-starter-kit-main/data/trec-2025-dragun-topics.jsonl
/kaggle/input/trec-2025/2025-starter-kit-main/modules/report_generator.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/segment_retriever.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/question_generator.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/query_generator.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/information_evaluator.py


In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth
    !pip install langchain langchain-community
    !pip install jsonlines
    !pip install --quiet langchain transformers sentencepiece

import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import json
from unsloth import FastLanguageModel
from unsloth import FastModel
import torch
from tqdm import tqdm
from unsloth.chat_templates import get_chat_template

2025-08-02 15:20:20.342584: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754148020.522907      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754148020.573710      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
TOPICS_PATH = '/kaggle/input/trec-2025/2025-starter-kit-main/data/trec-2025-dragun-topics.jsonl'

def load_jsonl(fname):
    with open(fname, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]
topics = load_jsonl(TOPICS_PATH)
print("Loaded", len(topics), "topics.")

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)  

from langchain.prompts import PromptTemplate

Loaded 30 topics.
==((====))==  Unsloth 2025.8.1: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [4]:
import re
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# 1. Enhanced Prompt with external resource-informed examples
prompt_template = PromptTemplate(
    input_variables=["title", "body"],
    template=(
        "You are an investigative fact-checking assistant trained on resources like PolitiFact, MBFC, and NewsQA.\n"
        "Your goal: Generate 10 sharp and critical questions a reader should ask to assess the credibility of a news article.\n\n"

        "Focus on:\n"
        "- Source bias or affiliations\n"
        "- Factual accuracy and evidence\n"
        "- Omissions or one-sided reporting\n"
        "- Language framing (sensationalism)\n"
        "- Viewpoint diversity (centrists, right-wing, etc.)\n"
        "- Potential conflicts of interest\n\n"

        "Guidelines:\n"
        "- Each question should be at most 300 characters.\n"
        "- Avoid compound questions (ask one thing at a time).\n"
        "- Avoid vague/general/overly broad questions.\n"
        "- Prioritize the most important and trust-assessing questions at the top.\n\n"

        "Example 1 (Inspired by PolitiFact):\n"
        "Title: 'WHO: Processed meat causes cancer'\n"
        "Article: WHO classified processed meat as carcinogenic...\n"
        "Questions:\n"
        "1. What peer-reviewed studies support WHO's claim?\n"
        "2. Is the research methodology disclosed in the article?\n"
        "3. Who funded the studies mentioned?\n"
        "4. Are dissenting expert views included?\n"
        "5. Are the health risks exaggerated?\n"
        "6. Is primary data linked or cited?\n"
        "7. Does the article mention affected demographics?\n"
        "8. Are any advocacy groups quoted?\n"
        "9. Is causal language justified?\n"
        "10. Are the claims corroborated by other agencies?\n\n"

        "Example 2 (MBFC-style analysis):\n"
        "Title: 'CDC says masks are optional outdoors'\n"
        "Article: CDC updates its outdoor masking guidance...\n"
        "Questions:\n"
        "1. Is there scientific consensus supporting this change?\n"
        "2. Does the article quote data or just opinion?\n"
        "3. Are counterarguments or dissenting experts cited?\n"
        "4. Are political motives or timing relevant?\n"
        "5. Is this consistent with international guidance?\n"
        "6. Does the article omit relevant prior CDC guidance?\n"
        "7. Are potential risks understated?\n"
        "8. Does the article rely on official or third-party data?\n"
        "9. Are specific regional risks acknowledged?\n"
        "10. Is the framing objective or emotionally loaded?\n\n"

        "Now write 10 similar questions for this article:\n\n"
        "Title: {title}\n"
        "Article text:\n{body}\n\n"
        "Output only the questions as a numbered list."
    )
)

# 2. LLM Pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=600,
    temperature=0.7,
    do_sample=True,
    return_full_text=False
)
llm = HuggingFacePipeline(pipeline=pipe)
llm_chain = LLMChain(prompt=prompt_template, llm=llm, output_key="text")

# 3. Regex pattern for clean extraction
pattern = re.compile(r"^\s*\d+[.)]\s*(.+)$")

# 4. Extract unique questions (≤300 chars)
def extract_unique_questions(raw_text):
    seen = set()
    questions = []
    for line in raw_text.strip().split("\n"):
        match = pattern.match(line.strip())
        if match:
            question = match.group(1).strip()
            if 0 < len(question) <= 300 and question not in seen:
                seen.add(question)
                questions.append(question)
    return questions

# 5. Main Processing Loop
results = []
for idx, topic in enumerate(topics):
    docid = topic['docid']
    title = topic.get('title', '').strip()
    body = topic.get('body', '').strip()[:2000]

    if not title and not body:
        print(f"⚠️ Skipping empty topic {docid}")
        continue

    questions = []
    for attempt in range(3):
        output = llm_chain.invoke({"title": title, "body": body})
        text = output['text'] if isinstance(output, dict) else output
        questions = extract_unique_questions(text)

        if len(questions) >= 10:
            questions = questions[:10]
            break
        else:
            print(f"🔁 Retry {attempt+1} for {docid} — Got {len(questions)}")

    while len(questions) < 10:
        questions.append("N/A")

    results.append({
        "docid": docid,
        "questions": questions
    })

# 6. Show Sample Output
for item in results[:3]:
    print(f"\nDocID: {item['docid']}")
    for i, q in enumerate(item['questions'], 1):
        print(f"{i}. {q}")


/tmp/ipykernel_36/3307494422.py:75: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
/tmp/ipykernel_36/3307494422.py:76: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt_template, llm=llm, output_key="text")


🔁 Retry 1 for msmarco_v2.1_doc_08_300872161 — Got 5
🔁 Retry 1 for msmarco_v2.1_doc_35_441326441 — Got 6
🔁 Retry 1 for msmarco_v2.1_doc_38_227081897 — Got 4
🔁 Retry 1 for msmarco_v2.1_doc_48_515287844 — Got 6

DocID: msmarco_v2.1_doc_04_420132660
1. What evidence supports Sam Panopoulos as the inventor of Hawaiian pizza?
2. Are there alternative claims about the origin of Hawaiian pizza not mentioned?
3. Does the article cite primary sources or rely on secondary accounts?
4. How does the article define "Hawaiian pizza" and its connection to Hawaii?
5. What sources confirm the 1962 date and location of invention?
6. Is there scholarly or historical research corroborating this origin story?
7. Does the article address cultural or regional variations in pizza naming?
8. Are there conflicting accounts from other restaurants or individuals?
9. How does the article handle the term "controversial" in describing Hawaiian pizza?
10. What role does the author’s affiliation with Gastro Obscura pla

In [5]:
import pandas as pd

team_id = "CUET"
run_id = "CUET-qwen14B-v1"

submission_rows = []

for item in results:
    topic_id = item["docid"]
    for rank, question in enumerate(item["questions"], 1):
        # Clean question of unwanted tabs/newlines
        clean_q = question.replace('\t', ' ').replace('\n', ' ').strip()
        submission_rows.append([
            topic_id,
            team_id,
            run_id,
            rank,
            clean_q
        ])

# Create DataFrame and save
df = pd.DataFrame(submission_rows, columns=["topic_id", "team_id", "run_id", "rank", "question"])
df.to_csv("/kaggle/working/CUET_run6.tsv", sep="\t", index=False, header=False, encoding="utf-8")

In [7]:
# Show duplicated questions (exact text matches)
duplicates = df[df.duplicated('question', keep=False)]
num_duplicates = len(duplicates)

print(f"🔍 Found {num_duplicates} duplicated question rows.\n")
if num_duplicates > 0:
    print("📄 Duplicated questions (full rows):")
    display(duplicates)

# Frequency count of duplicate questions
print("\n📊 Questions that appear more than once:")
duplicate_counts = df['question'].value_counts()
display(duplicate_counts[duplicate_counts > 1])

# Check for 'N/A' values (both literal string and NaN)
na_mask = df['question'].isna() | (df['question'].str.strip().str.upper() == 'N/A')
na_rows = df[na_mask]

print(f"\n⚠️ Found {len(na_rows)} rows with 'N/A' or missing question values.\n")
if len(na_rows) > 0:
    print("🚫 Rows with N/A or missing questions:")
    display(na_rows)


🔍 Found 0 duplicated question rows.


📊 Questions that appear more than once:


Series([], Name: count, dtype: int64)


⚠️ Found 0 rows with 'N/A' or missing question values.

